In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install torch
# !pip install torchvision
# !pip install segmentation-models-pytorch
# !pip install pycocotools
# !pip install transformers
# !pip install wandb
# !pip install colorama
# # !pip install ttach

# !pip install kaggle
# !mkdir /root/.kaggle
# !cp /content/drive/MyDrive/kaggle_competitions/kaggle.json /root/.kaggle  # kaggl.json위치 지정
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# !pip3 install openmim
# !mim install mmengine
# !mim install "mmcv>=2.0.0,<2.1.0"

# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# !pip install -e .

# import mmdet
# print(mmdet.__version__)

In [ ]:
import os
import sys
import gc
import re
import time
import datetime
import numpy as np

import pandas as pd
import random
import glob
import zlib
import base64
from datetime import datetime
from tqdm.notebook import tqdm
import json
import itertools
from itertools import chain
from types import SimpleNamespace
import wandb
import cv2
import PIL
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns

import albumentations as A
from albumentations.pytorch import ToTensorV2
# import ttach as tta

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import torchvision
import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold, StratifiedKFold

import pycocotools
import pycocotools.mask as mask_util
from pycocotools.coco import COCO

# package
sys.path.append("/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/detection_utils/")
import utils
from engine import train_one_epoch, evaluate

# color font
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [ ]:
# %cd /content/
# !kaggle competitions download -c hubmap-hacking-the-human-vasculature
# !unzip -qq /content/hubmap-hacking-the-human-vasculature.zip

# %cd /content/
# !cp /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/COCO_Dataset.zip /content/
# !unzip -qq /content/COCO_Dataset.zip

# if not os.path.exists(f'/content/mmdet_wheel/'):
#     os.mkdir( f'/content/mmdet_wheel/')

# %cd /content/mmdet_wheel/
# !cp /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/mmdet_wheel.zip /content/mmdet_wheel/
# !unzip -qq /content/mmdet_wheel/mmdet_wheel.zip

# %cd /content/

In [ ]:
from mmengine.config import Config
# base="mask-rcnn_r50_fpn_2x_coco"
base="mask-rcnn_x101-64x4d_fpn_2x_coco"
cfg = Config.fromfile(f'/content/mmdetection/configs/mask_rcnn/{base}.py')
dataset_type = 'CocoDataset'
classes = ('blood_vessel','glomerulus')
data_root = '/content'
num_classes = 2
img_size = 1024
batch_size = 4
seed = 42
#-----------------------------------------------------
cfg.model.roi_head.bbox_head.num_classes = num_classes
cfg.model.roi_head.mask_head.num_classes = num_classes
#-----------------------------------------------------
cfg.train_pipeline[2]['scale']=(img_size,img_size)
cfg.test_pipeline [1]['scale']=(img_size,img_size)
#-----------------------------------------------------
cfg.train_dataloader.dataset.type=dataset_type
cfg.train_dataloader.dataset.metainfo=dict(classes=classes)
cfg.train_dataloader.dataset.data_root=data_root
cfg.train_dataloader.dataset.ann_file='/content/coco_annotations_train_all_folds5_fold2.json'
cfg.train_dataloader.dataset.data_prefix=dict(img='train/')
cfg.train_dataloader.batch_size=batch_size
# cfg.train_dataloader.dataset.pipeline[2]['scale']=(img_size,img_size)
cfg.train_dataloader.dataset.pipeline = [
    {'type': 'LoadImageFromFile', 'backend_args': None},
    {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
    {'type': 'Resize', 'scale': (img_size, img_size), 'keep_ratio': True},
    {'type': 'RandomFlip', 'prob': 0.4, "direction":'vertical'},
    {'type': 'RandomFlip', 'prob': 0.4, "direction":'horizontal'},
    {'type': 'RandomShift', 'prob': 0.2},
    {'type': 'CutOut', "n_holes":3,  "cutout_shape": (img_size//12, img_size//12)},
    {'type': 'PhotoMetricDistortion',
                "brightness_delta":16,
                 "contrast_range":(0.8, 1.2),
                 "saturation_range":(0.8, 1.2),
                 "hue_delta":16},
    # {'type' : 'Normalize',"mean":[123.675, 116.28, 103.53],'std':[58.395, 57.12, 57.375], 'to_rgb':True},
    # {'type': 'Mosaic'},
    # {'type': 'MixUp'},
    {'type': 'PackDetInputs'}
]
#-----------------------------------------------------
cfg.val_dataloader.dataset.type=dataset_type
cfg.val_dataloader.dataset.metainfo=dict(classes=classes)
cfg.val_dataloader.dataset.data_root=data_root
cfg.val_dataloader.dataset.ann_file='/content/coco_annotations_valid_all_folds5_fold2.json'
cfg.val_dataloader.dataset.data_prefix=dict(img='train/')
cfg.val_dataloader.dataset.pipeline = [{'type': 'LoadImageFromFile', 'backend_args': None},
 {'type': 'Resize', 'scale': (img_size, img_size), 'keep_ratio': True},
#  {'type' : 'Normalize',"mean":[123.675, 116.28, 103.53],'std':[58.395, 57.12, 57.375], 'to_rgb':True},
 {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
 {'type': 'PackDetInputs',
  'meta_keys': ('img_id',
   'img_path',
   'ori_shape',
   'img_shape',
   'scale_factor')}]
#-----------------------------------------------------
cfg.test_dataloader.dataset.type=dataset_type
cfg.test_dataloader.dataset.metainfo=dict(classes=classes)
cfg.test_dataloader.dataset.data_root=data_root
cfg.test_dataloader.dataset.ann_file='/content/coco_annotations_valid_all_folds5_fold2.json'
cfg.test_dataloader.dataset.data_prefix=dict(img='train/')
cfg.test_dataloader.dataset.pipeline = [{'type': 'LoadImageFromFile', 'backend_args': None},
 {'type': 'Resize', 'scale': (img_size, img_size), 'keep_ratio': True},
#  {'type' : 'Normalize',"mean":[123.675, 116.28, 103.53],'std':[58.395, 57.12, 57.375], 'to_rgb':True},
 {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
 {'type': 'PackDetInputs',
  'meta_keys': ('img_id',
   'img_path',
   'ori_shape',
   'img_shape',
   'scale_factor')}]
#------------------------------------------------------
#------------------------------------------------------
cfg.val_evaluator.type='CocoMetric'
cfg.val_evaluator.ann_file='/content/coco_annotations_valid_all_folds5_fold2.json'
cfg.val_evaluator.metric=['segm']

cfg.test_evaluator.type='CocoMetric'
cfg.test_evaluator.ann_file='/content/coco_annotations_valid_all_folds5_fold2.json'
cfg.test_evaluator.metric=['segm']
#------------------------------------------------------
cfg.train_cfg.max_epochs=20
cfg.optim_wrapper.type='OptimWrapper'
cfg.optim_wrapper.optimizer=dict(type='AdamW',lr=5e-4,weight_decay=1e-2,eps=1e-8,betas=(0.9, 0.999))
cfg.lr_config = dict(
    policy='CosineAnnealing',
    warmup='linear',
    warmup_iters=300,
    warmup_ratio=1.0 / 10,
    min_lr_ratio=2e-5,
    by_epoch=False,
    step=5
    )
cfg.optimizer_config = dict(grad_clip=None)
cfg.optimizer_config.grad_clip = dict(max_norm=1000, norm_type=2)
cfg.default_hooks = dict(logger=dict(type='LoggerHook', interval=200),
                         checkpoint=dict(type='CheckpointHook', interval=1, save_best='coco/segm_mAP', max_keep_ckpts=1))
cfg.work_dir = f'/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/model/{base}_{img_size}/'
#------------------------------------------------------
!mkdir -p /content/configs/HuBMAP
config=f'/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/custom_config_{base}_{img_size}_{img_size}.py'
with open(config, 'w') as f:
    f.write(cfg.pretty_text)



In [ ]:
def seed_everthing(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_everthing(seed)

pd.set_option('display.max_columns', 200) ## 모든 열 출력
pd.set_option('display.max_rows', 200) ## 모든 행 출력

In [ ]:
!python /content/mmdetection/tools/train.py {config}

07/23 13:08:45 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
    CUDA available: True
    numpy_random_seed: 901876299
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_5